In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora, models
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import numpy as np
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq
from google.cloud import bigquery
from scipy import stats

In [3]:
# Create a BigQuery client
client = bigquery.Client.from_service_account_json(r'C:\Users\AkruthiSrikanth\Downloads\tech-cali-b2c-72b3e690e309-Compute-Engine.json')

# Define your BigQuery table details
project_id = 'tech-cali-b2c'
dataset_id = 'CE_Analytics_Layer'
table_id = 'Creator_Social_Profile'

# Create a reference to the BigQuery table
table_ref = client.dataset(dataset_id).table(table_id)

# Retrieve the data from BigQuery into a DataFrame
df_mod = client.query(f"SELECT * FROM `{project_id}.{dataset_id}.{table_id}`").to_dataframe()

In [4]:
df = df_mod.drop_duplicates(subset=['artist_id'], keep='first')
df = df[df['artist_id'] != ""]

In [5]:
df.iloc[:,3:] = df.iloc[:,3:].fillna('').replace('',0)

In [6]:
cols = ['Creator_ID', 'name', 'country_code', 'genres', 'artist_id']
columns_select = list(df.drop(cols, axis=1).columns)
for col in columns_select:
    df[col] = df[col].astype('float')

In [7]:
df_fi = df.copy()

In [8]:
# Label Encoding for Country code
le = LabelEncoder()
df_fi['country_code_label'] = le.fit_transform(df_fi['country_code'])

In [9]:
# Topic Modeling Code: Genre_Score (Topic distribution)
def get_genre_score(topic_distribution):
    most_likely_word = topic_distribution.argmax()
    genre_score = lda.components_[most_likely_word][0]
    return genre_score

# Create a TF-IDF vectorizer : Finds importance of the term
vectorizer = TfidfVectorizer(stop_words="english")

X = vectorizer.fit_transform(df_fi["genres"])

lda = LatentDirichletAllocation(n_components=10, random_state=0)

lda.fit(X)

topic_distributions = lda.transform(X)

major_genres = []
for i in range(len(df_fi)):
    topic_distribution = topic_distributions[i]
    major_genre = get_genre_score(topic_distribution)
    major_genres.append(major_genre)

# for i in range(len(data)):
#   print('Document: ',i, major_genres[i])

df_fi['genre_score'] = major_genres

In [10]:
# Topic Modeling (Major Genre Label):
genres_list = [genres.split(', ') for genres in df_fi['genres']]
dictionary = corpora.Dictionary(genres_list)
corpus = [dictionary.doc2bow(genres) for genres in genres_list]

num_topics = 1  # Number of topics you want to extract (in this case, major genre)
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

# Extract major genre for each row
major_genres = []
for genres in genres_list:
    genre_counts = {}
    for genre in genres:
      genre_counts[genre] = genre_counts.get(genre, 0) + 1
    major_genre = max(genre_counts, key=genre_counts.get)
    major_genres.append(major_genre)

df_fi['major_genre'] = major_genres

**Data Preprocessing**

In [11]:
df_rf = df_fi.drop(['Creator_ID', 'name', 'country_code', 'genres', 'major_genre', 'artist_id', 'youtube_daily_video_views','youtube_channel_views',
           'TikTok_Genz_followers_percentage','Instagram_Genz_followers_percentage','TikTok_number_of_top_tracks',
           'TikTok_avg_creations_for_top_n_tracks','TikTok_avg_views_for_top_n_tracks', 'Youtube_Genz_subscribers_percentage'], axis=1)
# df_rf = df_new.drop(df_new.filter(regex='_z_score|_z_score_label|_zscore'), axis=1)



df_filled = df_rf.fillna(df_rf.median())

In [12]:
df_filled.columns.tolist()

['spotify_followers',
 'spotify_monthly_listeners',
 'facebook_likes',
 'facebook_talks',
 'twitter_followers',
 'twitter_retweets',
 'youtube_monthly_video_views',
 'TikTok_followers',
 'TikTok_Genz_followers',
 'TikTok_avg_engagements_per_post',
 'TikTok_total_creations_for_top_n_tracks',
 'TikTok_total_views_for_top_n_tracks',
 'Instagram_followers',
 'Instagram_Genz_followers',
 'Instagram_avg_engagements_per_post',
 'Youtube_subscribers',
 'Youtube_Genz_subscribers',
 'Youtube_avg_engagements_per_post',
 'country_code_label',
 'genre_score']

**Standard Scaling**

In [13]:
scaler = StandardScaler()

scaler.fit(df_filled)

scaled_data = scaler.transform(df_filled)

column_names = scaler.get_feature_names_out()

scaled_df = pd.DataFrame(scaled_data, columns=column_names)

**Train Test Split**

In [14]:
# Separate features and target variable
X = scaled_df.drop('TikTok_total_creations_for_top_n_tracks', axis=1)
y = scaled_df['TikTok_total_creations_for_top_n_tracks']

# Train and test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

Training set size: (2917, 19)
Testing set size: (730, 19)


**Model Building and Hyperparameter Tuning**

In [15]:
# Function to calculate R-Square and RMSE
score_log = pd.DataFrame()
def score(model, name , x_test, y_test, position):
    y_pred = model.predict(x_test)
    score_log.loc[position, "Description"] = name
    score_log.loc[position, "R-Square"] = round(r2_score(y_test, y_pred), 3)
    score_log.loc[position,"MSE"] = round(mean_squared_error(y_test, y_pred), 3)
    score_log.loc[position, "RMSE"] = round(np.sqrt(mean_squared_error(y_test, y_pred)), 3)
    score_log.loc[position, "MAE"] = round(mean_absolute_error(y_test, y_pred), 3)
    return score_log

In [16]:
confidence_level = 0.95
def calculate_confidence_interval(scores):
  degrees_of_freedom = len(scores) - 1
  alpha = 1 - confidence_level

  mean_squared_error_mean = scores.mean()
  mean_squared_error_std = scores.std()
  t_critical = stats.t.ppf(1 - alpha / 2, degrees_of_freedom)

  margin_of_error = t_critical * (mean_squared_error_std / len(scores) ** 0.5)
  confidence_interval = (mean_squared_error_mean - margin_of_error, mean_squared_error_mean + margin_of_error)
  return confidence_interval

In [17]:
# Linear Regression
lr = LinearRegression()

scores = cross_val_score(lr, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
scores = abs(scores)

lr.fit(X_train, y_train)

confidence_interval = calculate_confidence_interval(scores)
print(f"Confidence Interval ({confidence_level * 100:.2f}%): {confidence_interval}")

Confidence Interval (95.00%): (0.2933460311293135, 0.684226181699465)


In [18]:
# Ridge Regression
ridge = Ridge()

scores = cross_val_score(ridge, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
scores = abs(scores)

ridge.fit(X_train, y_train)

confidence_interval = calculate_confidence_interval(scores)
print(f"Confidence Interval ({confidence_level * 100:.2f}%): {confidence_interval}")

Confidence Interval (95.00%): (0.29330382362480617, 0.6837780147129078)


In [19]:
# Lasso Regression
lasso = Lasso()

scores = cross_val_score(lasso, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
scores = abs(scores)

lasso.fit(X_train, y_train)

confidence_interval = calculate_confidence_interval(scores)
print(f"Confidence Interval ({confidence_level * 100:.2f}%): {confidence_interval}")

Confidence Interval (95.00%): (0.7064884710098477, 1.365129370858043)


In [20]:
# Decision Tree
dtr = DecisionTreeRegressor()

scores = cross_val_score(dtr, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
scores = abs(scores)

dtr.fit(X_train, y_train)

confidence_interval = calculate_confidence_interval(scores)
print(f"Confidence Interval ({confidence_level * 100:.2f}%): {confidence_interval}")

Confidence Interval (95.00%): (0.6057274433594396, 1.0473236015670095)


In [21]:
# SVR
svr = SVR(kernel="linear")

scores = cross_val_score(svr, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
scores = abs(scores)

svr.fit(X_train, y_train)

confidence_interval = calculate_confidence_interval(scores)
print(f"Confidence Interval ({confidence_level * 100:.2f}%): {confidence_interval}")

Confidence Interval (95.00%): (0.28865993296137027, 0.7104682852738894)


In [22]:
# Random Forest
rf_1 = RandomForestRegressor()
rf_1.fit(X_train, y_train)

scores = cross_val_score(rf_1, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
scores = abs(scores)

confidence_interval = calculate_confidence_interval(scores)
print(f"Confidence Interval ({confidence_level * 100:.2f}%): {confidence_interval}")

Confidence Interval (95.00%): (0.3403985802607266, 0.6961644365255766)


In [23]:
#Random Forest Tuning
param_grid = [{"n_estimators": [100, 200, 300, 400, 500],
              "max_depth": [7, 9, 12, 15],
              "min_samples_split": [5, 7, 12]}]

rf_t = RandomForestRegressor()
grid_cv = GridSearchCV(rf_t, param_grid, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)

best_estimator1 = grid_cv.best_estimator_

scores = cross_val_score(best_estimator1, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
scores = np.abs(scores)

confidence_interval = calculate_confidence_interval(scores)
print(f"Confidence Interval ({confidence_level * 100:.2f}%): {confidence_interval}")

Confidence Interval (95.00%): (0.3270265958386841, 0.6916172240901998)


In [24]:
#XGBoost
xg_boost = xgb.XGBRegressor()
xg_boost.fit(X_train, y_train)

scores = cross_val_score(xg_boost, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
scores = abs(scores)

confidence_interval = calculate_confidence_interval(scores)
print(f"Confidence Interval ({confidence_level * 100:.2f}%): {confidence_interval}")

Confidence Interval (95.00%): (0.31421672179506877, 0.6955908804393457)


In [25]:
#XGBoost Tuning

param_grid = [{'n_estimators': [100, 200, 300],
              'max_depth': [3, 5, 7, 9],
              'learning_rate': [0.01, 0.001]}]

xg_b_t = xgb.XGBRegressor()
grid_cv = GridSearchCV(xg_b_t, param_grid, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)

best_estimator2 = grid_cv.best_estimator_

scores = cross_val_score(best_estimator2, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
scores = np.abs(scores)

confidence_interval = calculate_confidence_interval(scores)
print(f"Confidence Interval ({confidence_level * 100:.2f}%): {confidence_interval}")

Confidence Interval (95.00%): (0.317762260842165, 0.7029088863524635)


In [26]:
score(lr, "LR - Test Error", X_test, y_test, 1)
score(ridge, "Ridge - Test Error", X_test, y_test, 2)
score(lasso, "Lasso - Test Error", X_test, y_test, 3)
score(dtr, "DT - Test Error", X_test, y_test, 4)
score(svr, "SVR - Test Error", X_test, y_test, 5)
score(rf_1, "RF - Before Tuning - Test Error", X_test, y_test, 6)
score(best_estimator1, "RF - After Tuning - Test error", X_test, y_test, 7)
score(xg_boost, "XGB - Before Tuning - Test Error", X_test, y_test, 8)
score(best_estimator2, "XGB - After Tuning - Test error", X_test, y_test, 9)

,Description,R-Square,MSE,RMSE,MAE
1,LR - Test Error,0.550,0.386,0.621,0.270
2,Ridge - Test Error,0.551,0.386,0.621,0.270
3,Lasso - Test Error,-0.002,0.860,0.927,0.480
4,DT - Test Error,0.353,0.555,0.745,0.313
5,SVR - Test Error,0.527,0.406,0.637,0.234
6,RF - Before Tuning - Test Error,0.517,0.415,0.644,0.268
7,RF - After Tuning - Test error,0.549,0.387,0.622,0.260
8,XGB - Before Tuning - Test Error,0.478,0.448,0.669,0.273
9,XGB - After Tuning - Test error,0.614,0.331,0.576,0.271


In [27]:
#Model with the highest R-Square
model_name = [lr, ridge, lasso, dtr, svr, rf_1, best_estimator1, xg_boost, best_estimator2]
r_square = score_log['R-Square']

zipped = zip(model_name,r_square)

best_model = max(zipped, key = lambda i : i[1])
model_name = best_model[0]

model_name

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

**Feature Importance**

In [28]:
# Sampled Shapley - manual coding
# def sampled_shapley(model, X, num_samples=1000):

#     if isinstance(X, pd.DataFrame):
#         X = X.values

#     num_instances, num_features = X.shape
#     feature_scores = np.zeros(num_features)

#     baseline = np.mean(model.predict(X))

#     for i in range(num_features):
#         # Initialize the cumulative sum of feature importance scores
#         cumulative_importance = 0.0

#         for j in range(num_samples):
#             # Generate a random permutation of the instance indices
#             perm = np.random.permutation(num_instances)

#             # Copy the original input data
#             X_perm = X.copy()

#             # Shuffle the values of the current feature according to the permutation
#             X_perm[:, i] = X_perm[perm, i]

#             # Calculate the prediction for the permuted data
#             prediction = np.mean(model.predict(X_perm))

#             # Calculate the contribution of the current feature to the prediction
#             contribution = abs(prediction - baseline)

#             # Accumulate the contribution to the cumulative sum
#             cumulative_importance += contribution

#         # Calculate the average contribution for the current feature
#         feature_scores[i] = cumulative_importance / num_samples

#     return feature_scores

In [32]:
# Sampled Shapley - using shap library
import shap
def model_predict(X):
    return model_name.predict(X)

explainer = shap.Explainer(model_predict, X)

shap_values = explainer.shap_values(X)

feature_scores = np.abs(shap_values).mean(axis=0)

Permutation explainer: 3648it [05:14, 11.41it/s]


In [33]:
# feature_scores = sampled_shapley(model_name, X_test, num_samples=1000)

feature_imp = pd.DataFrame([feature_scores], columns = X.columns)

feature_imp = feature_imp.reset_index(drop=True)

feature_imp

,spotify_followers,spotify_monthly_listeners,facebook_likes,facebook_talks,twitter_followers,twitter_retweets,youtube_monthly_video_views,TikTok_followers,TikTok_Genz_followers,TikTok_avg_engagements_per_post,TikTok_total_views_for_top_n_tracks,Instagram_followers,Instagram_Genz_followers,Instagram_avg_engagements_per_post,Youtube_subscribers,Youtube_Genz_subscribers,Youtube_avg_engagements_per_post,country_code_label,genre_score
0,0.019754,0.036529,0.001878,0.002828,0.000821,0.000128,0.006626,0.004968,0.004168,0.000721,0.337056,0.015879,0.007411,0.002889,0.000059,0.016953,0.00023,0.003898,0.012735


In [34]:
# Define your BigQuery project ID and credentials
project_id = 'tech-cali-b2c'
credentials = service_account.Credentials.from_service_account_file(r'C:\Users\AkruthiSrikanth\Downloads\tech-cali-b2c-72b3e690e309-Compute-Engine.json')

# Define the BigQuery table name and dataset ID
dataset_id = 'CE_ML_Layer'
table_name = 'Feature_Importance'

# Write the dataframe to BigQuery
pandas_gbq.to_gbq(feature_imp, f'{dataset_id}.{table_name}', project_id=project_id, if_exists='replace', credentials=credentials, progress_bar=True)

print('Data moved to BigQuery successfully!')

100%|██████████| 1/1 [00:00<00:00, 3625.15it/s]

Data moved to BigQuery successfully!
